DOWNLOAD DATA FROM https://github.com/abachaa/MTS-Dialog. then add it to the runtimes files.

In [ ]:
!pip install datasets evaluate transformers rouge-score nltk
!pip install bert_score
!pip install transformers --upgrade

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9fff8bfb565ea2f0ee1afc65d5a5fae0c2aac8fd7f59ccac7cae5fcb8794fe13
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dep

### IMPORTS


In [ ]:
import nltk
import torch
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoConfig
from datasets import load_dataset
from evaluate import load
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from bert_score import score
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
from transformers import DataCollatorForLanguageModeling
from transformers import BartConfig, BartForConditionalGeneration, AutoTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


### Load the models


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
metric = load("rouge")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

## Prestrain with XSUM first


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset
import numpy as np
import evaluate

model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print("Loading XSum dataset...")
xsum_dataset = load_dataset("xsum")

def preprocess_xsum_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_xsum = xsum_dataset.map(preprocess_xsum_function, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

rouge_score = evaluate.load("rouge")


In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds[preds == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip().split(". ")) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split(". ")) for label in decoded_labels]

    result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

xsum_training_args = Seq2SeqTrainingArguments(
    output_dir="./xsum_finetuned_model",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

xsum_trainer = Seq2SeqTrainer(
    model=model,
    args=xsum_training_args,
    train_dataset=tokenized_xsum["train"],
    eval_dataset=tokenized_xsum["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train on XSum
print("Pre-finetuning on XSum...")
xsum_trainer.train()

# Save the XSum pre-finetuned model
model.save_pretrained("./xsum_finetuned_model")
tokenizer.save_pretrained("./xsum_finetuned_model")

###  Load MTS Dialog


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/xsum_finetuned_model")
tokenizer = AutoTokenizer.from_pretrained("/content/xsum_finetuned_model")
import pandas as pd
df1 = pd.read_csv('/content/MTS-Dialog-Augmented-TrainingSet-1-En-FR-EN-2402-Pairs.csv')
df2 = pd.read_csv('/content/MTS-Dialog-Augmented-TrainingSet-2-EN-ES-EN-2402-Pairs.csv')
df3 = pd.read_csv('/content/MTS-Dialog-TrainingSet.csv')

combined_df = pd.concat([df1, df2, df3], ignore_index=True)
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

shuffled_df.to_csv('combined_shuffled_file.csv', index=False)

train_dataset = load_dataset("csv", data_files="combined_shuffled_file.csv")
validation_dataset = load_dataset("csv", data_files="/content/MTS-Dialog-ValidationSet.csv")
metric = load("rouge")

Generating train split: 0 examples [00:00, ? examples/s]

# Preprocess training data into functions, including the Header ID in inputs

In [ ]:
def preprocess_function(examples, max_source_length=512, max_target_length=128):
    inputs = [f"{header} {dialogue}" for header, dialogue in zip(examples["section_header"], examples["dialogue"])]
    targets = [summary for summary in examples["section_text"]]


    # Decide on your padding strategy (dynamic is often better)
    padding = False  # Let the collator handle it

    # Get tokenized inputs with attention mask
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Get tokenized targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # Handle pad tokens in loss
    model_inputs["labels"] = labels["input_ids"]
    if padding == "max_length":  # If using static padding
        model_inputs["labels"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    return model_inputs

tokenized_train = train_dataset["train"].map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset["train"].column_names
)
tokenized_val = validation_dataset["train"].map(
    preprocess_function,
    batched=True,
    remove_columns=validation_dataset["train"].column_names
)


Map:   0%|          | 0/6005 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Initisalist data collator

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)


Add compute metric function to evaluate during training

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions[predictions == -100] = tokenizer.pad_token_id

    # Decode generated summaries and remove special tokens
    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Replace -100 in the labels (used for ignored positions) with pad token id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries
    decoded_labels = tokenizer.batch_decode(
        labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute BERTScore
    P, R, F1 = score(decoded_preds, decoded_labels, lang="en", rescale_with_baseline=True)

    # Convert PyTorch tensors to float values
    precision = P.mean().item()
    recall = R.mean().item()
    f1 = F1.mean().item()

    # Also compute ROUGE for comparison
    rouge_results = metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # Format the results
    results = {
        "bertscore_precision": round(precision * 100, 2),
        "bertscore_recall": round(recall * 100, 2),
        "bertscore_f1": round(f1 * 100, 2),
    }

    # Add ROUGE scores for comparison
    for k, v in rouge_results.items():
        results[k] = round(v * 100, 2)

    # Calculate prediction lengths for analysis
    prediction_lens = [len(tokenizer.encode(pred)) for pred in decoded_preds]
    results["gen_len"] = np.mean(prediction_lens)

    return results

Initialise training hyperparameters

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    generation_max_length=256,
    generation_num_beams=4,
    metric_for_best_model="bertscore_f1",
    greater_is_better=True,
    gradient_accumulation_steps=2,
    warmup_ratio=0.15,
    fp16=True,
    lr_scheduler_type="cosine",
)
# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-39-d7954182d782>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Start training

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Bertscore Precision,Bertscore Recall,Bertscore F1,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.708400,2.185866,39.820000,34.070000,36.720000,36.690000,14.430000,29.480000,29.590000,30.780000
2,0.779000,2.403757,45.370000,29.060000,36.940000,34.210000,12.940000,29.220000,29.030000,22.780000
3,0.403500,2.538575,44.850000,35.480000,40.020000,40.010000,14.980000,32.260000,32.330000,32.230000
4,0.211600,2.701327,43.560000,40.640000,41.960000,43.450000,17.630000,34.710000,34.660000,38.190000
5,0.093000,2.818460,41.600000,37.810000,39.550000,40.080000,15.610000,31.970000,32.000000,38.650000


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

TrainOutput(global_step=9006, training_loss=0.7468774742532671, metrics={'train_runtime': 1509.1444, 'train_samples_per_second': 23.874, 'train_steps_per_second': 5.968, 'total_flos': 4345425013063680.0, 'train_loss': 0.7468774742532671, 'epoch': 5.996336996336996})

Save the model


In [ ]:
trainer.save_model("bart_ce_finetuned")

Generate summaries from the test dataset

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
from transformers import pipeline
from evaluate import load
import nltk
from tqdm import tqdm

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Load your fine-tuned model and its corresponding tokenizer
model_name = 'bart_ce_finetuned'
pipe = pipeline('summarization', model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the evaluation metric
rouge = load("rouge")

# Generation parameters
gen_kwargs = {
    'num_beams': 12,
    'length_penalty': 0.6,
    'max_length': 128,
}

# Load the validation dataset
validation_data = pd.read_csv('/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv')

# Process each dialogue using the same preprocessing as training
dialogues = []
predicted_summaries = []
ground_truth_summaries = []
rouge_scores = []

for i, row in tqdm(validation_data.iterrows(), total=len(validation_data), desc="Evaluating dialogues"):
    # Check if the required columns exist
    if 'section_header' in row and 'dialogue' in row and 'section_text' in row:
        # Apply the same preprocessing as during training
        input_text = f"{row['section_header']} {row['dialogue']}"
        ground_truth = f"{row['section_text']}"
    else:
        # Fallback if columns don't match
        input_text = row['dialogue']
        ground_truth = row['section_text']

    # Generate summary
    summary_output = pipe(input_text, **gen_kwargs)
    predicted_summary = summary_output[0]['summary_text']

    # Store results
    dialogues.append(row['dialogue'])
    predicted_summaries.append(predicted_summary)
    ground_truth_summaries.append(ground_truth)

    # Compute ROUGE scores
    score = rouge.compute(predictions=[predicted_summary], references=[ground_truth])
    rouge_scores.append(score)

# Calculate average ROUGE scores
avg_rouge = {metric: sum(score[metric] for score in rouge_scores) / len(rouge_scores)
             for metric in rouge_scores[0].keys()}
print("Average ROUGE scores:", avg_rouge)

# Create a results dataframe
results_df = pd.DataFrame({
    'dialogue': dialogues,
    'ground_truth': ground_truth_summaries,
    'prediction': predicted_summaries,
    'rouge_l': [score['rougeL'] for score in rouge_scores]
})

# Save results to CSV
results_df.to_csv('model_evaluation_results.csv', index=False)
print("\nDetailed results saved to 'model_evaluation_results.csv'")

Device set to use cuda:0
Evaluating dialogues: 100%|██████████| 200/200 [02:11<00:00,  1.52it/s]

Average ROUGE scores: {'rouge1': np.float64(0.35981035385299387), 'rouge2': np.float64(0.1662264640467998), 'rougeL': np.float64(0.2942862546667049), 'rougeLsum': np.float64(0.2942862546667049)}

Detailed results saved to 'model_evaluation_results.csv'


Use labels in test dataset to evaluate using Bert F1 and rouge

In [ ]:
P, R, F1 = score(predicted_summaries, ground_truth_summaries, lang="en", rescale_with_baseline=True)
rouge_results = metric.compute(
      predictions=predicted_summaries,
      references=ground_truth_summaries,
      use_stemmer=True
    )
print(P.mean())
print(R.mean())
print(F1.mean())


Evaluating dialogues: 100%|██████████| 200/200 [00:00<00:00, 21612.88it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.3315)
tensor(0.3490)
tensor(0.3385)
{'rouge1': np.float64(0.37561837912747364), 'rouge2': np.float64(0.170021136425291), 'rougeL': np.float64(0.3052723897707921), 'rougeLsum': np.float64(0.3039747549713749)}

10 Lowest BERTScore F1 Indices:
1. Index 15: F1 = -0.1647
2. Index 39: F1 = -0.1360
3. Index 151: F1 = -0.1184
4. Index 55: F1 = -0.0552
5. Index 77: F1 = -0.0339
6. Index 32: F1 = -0.0312
7. Index 88: F1 = -0.0110
8. Index 114: F1 = -0.0062
9. Index 83: F1 = 0.0025
10. Index 14: F1 = 0.0234


NameError: name 'rouge_l_scores' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!cp -r bart_ce_finetuned /content/drive/MyDrive/bart_ce_finetuned
!cp -r model_evaluation_results.csv /content/drive/MyDrive/model_evaluation_results.csv